In [ ]:
%%bigquery
DROP TABLE IF EXISTS `sevis-beta.sevis_staging.opt_magnitude_plot2`;
  CREATE OR REPLACE TABLE `sevis-beta.sevis_staging.opt_magnitude_plot2` AS
  WITH prep AS (
    SELECT
      `Year` AS fiscal_year,
      DATE(Year - 1, 10, 1) AS fy_start,
      DATE(Year, 9, 30) AS fy_end,
      Employment_OPT_Type,
      Employer_State,
      EMPLOYER_LMA,
      Employment_Description,
      Campus_State,
      CAMPUS_LMA,
      Student_Edu_Level_Desc,
      IS_STEM,
      NSF_SUBJ_FIELD_BROAD,
      Country_of_Birth,
      School_Name,
      SEVIS_ID,
      SAFE_CAST(Program_Start_Date AS DATE) AS psd,
      SAFE_CAST(Authorization_Start_Date AS DATE) AS auth_start_date,
      SAFE_CAST(Authorization_End_Date AS DATE) AS auth_end_date
    FROM `sevis-beta.sevis_raw.sevis_f1_cleaned_master`
  )
  SELECT
    fiscal_year,
    Employer_State,
    EMPLOYER_LMA,
    Campus_State,
    CAMPUS_LMA,
    Student_Edu_Level_Desc,
    IS_STEM,
    NSF_SUBJ_FIELD_BROAD,
    Country_of_Birth,
    School_Name,
    SEVIS_ID
  FROM prep
  WHERE
        Employment_Description = 'opt'
    AND Employment_OPT_Type IN ('stem', 'post-completion')
    AND auth_start_date IS NOT NULL
    AND auth_end_date >= fy_start
    AND auth_start_date <= fy_end
    AND auth_start_date > psd;

Query is running:   0%|          |

""


In [ ]:
%%bigquery
-- standardSQL
/* ───── 1. Declare the variables your WHERE‑clauses expect ───── */
DECLARE fys ARRAY<INT64>;
DECLARE emp_states  ARRAY<STRING>;
DECLARE campus_states ARRAY<STRING>;
DECLARE edu_levels  ARRAY<STRING>;
DECLARE stem_param     BOOL;
DECLARE nsf_fields ARRAY<STRING>;
DECLARE birth_countries    ARRAY<STRING>;
DECLARE schools ARRAY<STRING>;
DECLARE emp_lmas ARRAY<STRING>;
DECLARE campus_lmas ARRAY<STRING>;

/* ───── 2. Assign test values ───── */
-- I'd like NULL values to be assigned to an "Other" category,
SET fys = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022];
SET emp_states  = NULL;--['massachusetts'];               -- NULL for all states
SET campus_states = NULL;
--#['alabama','alaska','arizona','arkansas','california','colorado','connecticut','delaware','district of columbia','florida','georgia','guam','hawaii','idaho','illinois','indiana','iowa','kansas','kentucky','louisiana','maine','maryland','massachusetts','michigan','minnesota','mississippi','missouri','montana','nebraska','nevada','new hampshire','new jersey','new mexico','new york','north carolina','north dakota','northern mariana islands','ohio','oklahoma','oregon','pennsylvania','puerto rico','rhode island','south carolina','south dakota','tennessee','texas','utah','vermont','virgin islands of the us','virginia','washington','west virginia','wisconsin','wyoming'];                      -- keep NULL when unused
SET edu_levels  = NULL;--['masters'];
SET stem_param     = NULL;                      -- NULL → both STEM & non‑STEM
SET nsf_fields = NULL;
SET birth_countries = NULL;--['tuvalu'];--NULL;
SET schools = NULL;--['a f international school of languages inc'];
SET emp_lmas = NULL;
SET campus_lmas = NULL;

/* ───── 3. Query to generate a table for the first plot ───── */
-- in a each given year, the bottom bar should be the grads_opt_total and the top one should be the non_opt_total
SELECT
    fiscal_year AS year,
    COUNT(DISTINCT SEVIS_ID) AS opt_student_count_total
FROM `sevis-beta.sevis_staging.opt_magnitude_plot2`
WHERE
      (fys IS NULL OR fiscal_year     IN UNNEST(fys))
  AND (campus_states IS NULL OR Campus_State           IN UNNEST(campus_states))
  AND (emp_states IS NULL OR Employer_State           IN UNNEST(emp_states))
  AND (edu_levels  IS NULL OR Student_Edu_Level_Desc IN UNNEST(edu_levels))
  AND (stem_param     IS NULL OR IS_STEM                =  stem_param)
  AND (nsf_fields IS NULL OR NSF_SUBJ_FIELD_BROAD IN UNNEST(nsf_fields))
  AND (birth_countries    IS NULL OR Country_of_Birth IN UNNEST(birth_countries))
  AND (schools IS NULL OR School_Name IN UNNEST(schools))
  AND (campus_lmas IS NULL OR CAMPUS_LMA IN UNNEST(campus_lmas))
  AND (emp_lmas IS NULL OR EMPLOYER_LMA IN UNNEST(emp_lmas))
GROUP BY fiscal_year
ORDER BY fiscal_year;

Query is running:   0%|          |

Downloading:   0%|          |

,year,opt_student_count_total
0,2010,128136
1,2011,143588
2,2012,160115
3,2013,189441
4,2014,211601
5,2015,244303
6,2016,288519
7,2017,330478
8,2018,348940
9,2019,354900


In [ ]:

# %%bigquery
# DROP TABLE IF EXISTS `sevis-beta.supporting_data.plot2test`;
# CREATE OR REPLACE TABLE `sevis-beta.supporting_data.plot2test` AS
# WITH prep AS (
#   SELECT
#     `Year`                                   AS fiscal_year,
#     /* fiscal-year window */
#     DATE(Year - 1, 10,  1)                  AS fy_start,   -- Oct-01 of prev year
#     DATE(Year    ,  9, 30)                  AS fy_end,     -- Sep-30 of same year

#     Employment_OPT_Type,             -- requested breakdown fields
#     Employer_State,
#     EMPLOYER_LMA,
#     Employment_Description,

#     Campus_State,
#     CAMPUS_LMA,
#     Student_Edu_Level_Desc,

#     IS_STEM,
#     NSF_SUBJ_FIELD_BROAD,

#     Country_of_Birth,
#     School_Name,

#     SEVIS_ID,

#     SAFE_CAST(Authorization_Start_Date AS DATE) AS auth_start_date,
#     SAFE_CAST(Authorization_End_Date AS DATE) AS auth_end_date
#   FROM `sevis-beta.sevis_raw.sevis_f1_cleaned_master`
# )
# , eligible AS (
#   SELECT
#     fiscal_year,

#     Employer_State,
#     EMPLOYER_LMA,

#     Campus_State,
#     CAMPUS_LMA,
#     Student_Edu_Level_Desc,

#     IS_STEM,
#     NSF_SUBJ_FIELD_BROAD,

#     Country_of_Birth,
#     School_Name,

#     SEVIS_ID
#   FROM prep
#   WHERE
#         Employment_Description = 'opt'
#     AND Employment_OPT_Type IN ('stem', 'post-completion')
#     AND auth_start_date IS NOT NULL
#     AND auth_end_date   >= fy_start      -- ends on/after FY start
#     AND auth_start_date <= fy_end        -- starts on/before FY end
# )
# SELECT
#   fiscal_year,

#   Employer_State,
#   EMPLOYER_LMA,

#   Campus_State,
#   CAMPUS_LMA,
#   Student_Edu_Level_Desc,

#   IS_STEM,
#   NSF_SUBJ_FIELD_BROAD,

#   Country_of_Birth,
#   School_Name,

#   COUNT(DISTINCT SEVIS_ID)         AS opt_student_count
# FROM eligible
# GROUP BY
#   fiscal_year,
#   Employer_State,
#   EMPLOYER_LMA,

#   Campus_State,
#   CAMPUS_LMA,
#   Student_Edu_Level_Desc,

#   IS_STEM,
#   NSF_SUBJ_FIELD_BROAD,

#   Country_of_Birth,
#   School_Name
# ORDER BY
#   fiscal_year,
#   Employer_State,
#   EMPLOYER_LMA,

#   Campus_State,
#   CAMPUS_LMA,
#   Student_Edu_Level_Desc,

#   IS_STEM,
#   NSF_SUBJ_FIELD_BROAD,

#   Country_of_Birth,
#   School_Name;